In [1]:
import time
import pickle 
import numpy as np 
import pandas as pd
from juliacall import Main as jl  
from hdna import *

[juliapkg] Locating Julia ^1.6.1
[juliapkg] Using Julia 1.8.5 at /home/marco/julia/julia-1.8.5/bin/julia
[juliapkg] Using Julia project at /home/marco/miniconda3/julia_env
[juliapkg] Installing packages:
           julia> import Pkg
           julia> Pkg.add([Pkg.PackageSpec(name="IJulia", uuid="7073ff75-c697-5162-941a-fcdaad2a7d2a"), Pkg.PackageSpec(name="PythonCall", uuid="6099a3de-0909-46bc-b1f4-468b9a2dfc0d")])
           julia> Pkg.resolve()


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/miniconda3/julia_env/Project.toml`
  [7073ff75] + IJulia v1.24.0
  [6099a3de] + PythonCall v0.9.10
    Updating `~/miniconda3/julia_env/Manifest.toml`
  [8f4d0f93] + Conda v1.7.0
  [992eb4ea] + CondaPkg v0.2.15
  [9a962f9c] + DataAPI v1.14.0
  [e2d170a0] + DataValueInterfaces v1.0.0
  [7073ff75] + IJulia v1.24.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [692b3bcd] + JLLWrappers v1.4.1
  [682c06a0] + JSON v0.21.3
  [0f8b85d8] + JSON3 v1.12.0
  [1914dd2f] + MacroTools v0.5.10
  [739be429] + MbedTLS v1.1.7
  [0b3b1443] + MicroMamba v0.1.12
  [bac558e1] + OrderedCollections v1.4.1
  [69de0a69] + Parsers v2.5.3
  [fa939f87] + Pidfile v1.3.0
  [21216c6a] + Preferences v1.3.0
  [6099a3de] + PythonCall v0.9.10
  [ae029012] + Requires v1.3.0
  [6c6a2e73] + Scratch v1.1.1
  [66db9d55] + SnoopPrecompile v1.0.3
  [b85f4697] + SoftGlobalScope v1.1.0
  [856f2bd8] + StructTypes v1.10

In [2]:
expdata = pd.read_csv('./data/herteldata.csv')

In [3]:
rates = []

model = Model('dna', '3D')

data = expdata.iloc[0]

print(f'Simulating string {(data[0]).strip()}')
A = Strand(model, str(data[0]).strip())
B = A.complementary()
kinet = Kinetwork(model, A, B, 3)
geo = Geometry(180, 360)
K = Kinetics(model, kinet, geo)
opts = Options(method='direct', runtime=2e-6, Nmonte=10000)
simulatore = Simulator(model, kinet, K, options=opts)

Simulating string GTTCGGTCTA


In [4]:
%time sim1 = simulatore.ensemble()

100%|██████████| 10000/10000 [00:13<00:00, 731.16it/s]

CPU times: user 12.2 s, sys: 2.02 s, total: 14.2 s
Wall time: 14.2 s


In [6]:
%time m = simulatore.mfpts(sim1)

2275 simulations didn't produce a duplex.
That's 22.75% of simulations
CPU times: user 1.19 s, sys: 1.25 ms, total: 1.19 s
Wall time: 1.19 s


In [7]:
print('{:e}'.format(1/m))

1.545742e+06


In [13]:
jl.findfirst(jl.isone, sim1[0][-1,:])

26

In [ ]:
simulatore.Graph.edges['..........+..........','....(((...+...)))....']

{'kind': 'on_nucleation', 'kf': 238251003.38199154, 'kb': 33164.84852282317}

In [ ]:
print(f"['{A.sequence}+{B.invert.sequence}',]")
print(simulatore.Graph.edges['.......(((+))).......','......((((+))))......'])
print('{:e}'.format(simulatore.Graph.edges['.......(((+))).......','......((((+))))......']['kf']))
print('{:e}'.format(simulatore.Graph.edges['.......(((+))).......','......((((+))))......']['kb']))
simulatore.trajectory(df)

['GTTCGGTCTA+TAGACCGAAC',]
{'kind': 'zipping', 'kf': 7624032108.223729, 'kb': 1915947574.5110023}
7.624032e+09
1.915948e+09


['..........+..........',
 '......(((.+.)))......',
 '.....(((((+))))).....',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((+))).......',
 '......((((+))))......',
 '.......(((

In [ ]:
import time
index = 0
print(len(E[index].t))
%time np.argmax(jl.hcat(*E[index].u).__array__()[-1])

202
CPU times: user 2.73 ms, sys: 379 µs, total: 3.11 ms
Wall time: 1.79 ms


12

In [ ]:
fpi = []
for index in range(110):
    d = jl.hcat(*E[index].u).__array__()[-1]


KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm
fpi = []
for index in tqdm(range(2000)):
    d = [E[index].u[i][-1] for i in range(len(E[index].u))]
    fpi.append(np.argmax(d))


 60%|█████▉    | 1194/2000 [00:19<02:12,  6.09it/s] 

: 

: 

In [ ]:
rates = []

model = Model('dna', '3D')

for i, data in expdata.iterrows():
    print(f'Simulating string {(data[0]).strip()}')
    A = Strand(model, str(data[0]).strip())
    B = A.complementary()
    %time kinet = Kinetwork(model, A, B, 3)
    geo = Geometry(180, 360)
    K = Kinetics(model, kinet, geo)
    opts = Options(method='direct', runtime=2e-6, Nmonte=2000)
    simulatore = Simulator(model, kinet, K, options=opts)
    E = simulatore.ensemble()
    tau = simulatore.mfpts(E)
    rates.append(1/tau)
    print(f"Experimental rate:  {'{:e}'.format(data[1])}")
    print(f"Predicted rate:     {'{:e}'.format(1/tau)}")
    if i == 0:
        break

Simulating string GTTCGGTCTA
CPU times: user 1.92 s, sys: 59.9 ms, total: 1.98 s
Wall time: 1.94 s


  2%|▏         | 34/2000 [00:22<21:59,  1.49it/s]


JuliaError: "an error occurred while setting an error"

In [ ]:
expdata['calc'] = rates

In [ ]:
expdata

,Seqs:,Expt:,calc
0,GTTCGGTCTA,1145949.98,78005.723739
1,CCAAAACCAA,1668153.61,81305.291990
2,AAACCACACA,1809459.04,90420.280870
3,ACCAAACCAC,2167674.56,139004.885069
4,ACACCAAACC,2178513.20,110601.715691
5,AAAAACCCAC,2364640.37,166230.023928
6,CAACACCCAA,2511513.09,121725.173379
7,AAACCCACCA,2527035.41,137248.059320
8,ACAACACCAC,2559279.25,153427.373333
9,CAAAACCCCA,2651666.39,96282.638320


In [ ]:
model = Model('dna', '3D')
A = Strand(model, str('AAAAAAAAAAAAA').strip())
B = A.complementary()
kinet = Kinetwork(model, A, B, 3)
geo = Geometry(120, 120)
K = Kinetics(model, kinet, geo)

In [ ]:
dl = K.diffusionlimited()
geo = K.geometric_rate()
print('{:e}'.format(dl))
print('{:e}'.format(geo))